In [4]:
from ultralytics import YOLO

In [ ]:
from ultralytics import YOLO
import os
import cv2

model = YOLO("/home/minhpn/Desktop/Green_Parking/Model_training/YOLOv11_Detect_Number_From_Plate/content/runs/detect/train5/weights/best.pt")  # Update this

image_folder = "/home/minhpn/Desktop/Green_Parking/Model_training/YOLO_to_labelme_func/train/images"
output_folder = "/home/minhpn/Desktop/Green_Parking/Model_training/YOLO_to_labelme_func/train/temp_txt"
os.makedirs(output_folder, exist_ok=True)

# Loop
for img_file in os.listdir(image_folder):
    if img_file.lower().endswith(".jpg"):
        img_path = os.path.join(image_folder, img_file)
        print(img_path)
        image = cv2.imread(img_path)
        if image is None:
            print(f"Skipped {img_file} (could not read)")
            continue

        h, w = image.shape[:2]

        # Run YOLO prediction
        results = model.predict(source=img_path, conf=0.25, save=False, show=False)

        # Save prediction in YOLO txt format
        label_path = os.path.join(output_folder, os.path.splitext(img_file)[0] + ".txt")

        with open(label_path, "w") as f:
            for box in results[0].boxes:
                cls_id = int(box.cls[0])
                x1, y1, x2, y2 = box.xyxy[0]

                x_center = ((x1 + x2) / 2) / w
                y_center = ((y1 + y2) / 2) / h
                width = (x2 - x1) / w
                height = (y2 - y1) / h

                f.write(f"{cls_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")

        print(f"Saved: {label_path}")
